# Data Lake Project

Author: Karim Ahmed


This notebook runs on AWS EMR cluster in the `us-west-2` region using pyspark kernel.
https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-managed-notebooks.html

In [ ]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import (
    year,
    month,
    dayofmonth,
    hour,
    weekofyear,
    dayofweek,
    date_format,
    monotonically_increasing_id,
)

In [ ]:
# Create Spark session
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .getOrCreate()

In [ ]:
# Input and output S3s

input_data = "s3a://udacity-dend/"
output_data = "s3a://aws-bucket-udacity-de/"

## Process song data

Create and store `songs_table` and `artists_table`

In [ ]:
# read song data file
song_data = input_data + "song_data/*/*/*/*.json"
song_df = spark.read.json(song_data)

In [ ]:
# extract columns to create songs table
# song_id, title, artist_id, year, duration
songs_table = song_df.select(
    'song_id', 'title', 'artist_id', 'year', 'duration')

# write songs table to parquet files partitioned by year and artist
songs_table.write.mode("overwrite").partitionBy(
    "artist_id", "year").parquet(output_data + "/songs")

In [ ]:
# extract columns to create artists table
# artist_id, name, location, lattitude, longitude
artists_table = song_df.select(
    'artist_id',
    'artist_name',
    'artist_location',
    'artist_latitude',
    'artist_longitude',
).dropDuplicates()

# write artists table to parquet files
artists_table.write.mode("overwrite").parquet(output_data + "/artists")

## Process log data

Create and store `users_table` and `time_table`

In [ ]:
log_data = input_data + "log_data/*/*/*.json"

# read log data file
log_df = spark.read.json(log_data)

In [ ]:
# filter by actions for song plays
log_df = log_df.filter(log_df.page=="NextSong")

In [ ]:
# extract columns for users table 
# user_id, first_name, last_name, gender, level
users_table = log_df.select(
    col("userId"),
    col("firstName"),
    col("lastName"),
    col("gender"),
    col("level"),
).dropDuplicates()

# write users table to parquet files
users_table.write.mode("overwrite").parquet(output_data + "/users")

In [ ]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.fromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S'))
log_df = log_df.withColumn('timestamp', get_timestamp(log_df.ts))

# create datetime column from original timestamp column
get_datetime = udf(lambda x: str(datetime.fromtimestamp(int(x) / 1000.0)))
log_df = log_df.withColumn('datetime', get_datetime(log_df.ts))

In [ ]:
# extract columns to create time table
time_table = log_df.select('timestamp).withColumn(
    'start_time', lod_df.timestamp
).withColumn(
    'hour', dayofmonth('timestamp')
).withColumn(
    'day', dayofmonth('timestamp')
).withColumn(
    'week', weekofyear('timestamp')
).withColumn(
    'month', month('timestamp')
).withColumn(
    'year', year('timestamp')
).withColumn(
    'weekday', dayofweek('timestamp')
).dropDuplicates()

# write time table to parquet files partitioned by year and month
time_table.write.partitionBy(
        "year", "month").mode("overwrite").parquet(output_data + "/time_table")

## Create Songplays_table

In [ ]:
# extract columns from joined song and log datasets
# to create songplays table
log_df = log_df.alias('log_df')
song_df = song_df.alias('song_df')
songplays_table = log_df.join(
    song_df, col('log_df.artist') == col('song_df.artist_name')).select(
    col("log_df.datetime").alias("start_time"),
    col("log_df.userId").alias("user_id"),
    col("log_df.level"),
    col("song_df.song_id"),
    col("song_df.artist_id"),
    col("log_df.sessionId").alias("session_id"),
    col("log_df.location").alias("location"),
    col("log_df.userAgent").alias("user_agent"),
    month('log_df.datetime').alias('month'),
    year('log_df.datetime').alias('year'),
).withColumn('songplay_id', monotonically_increasing_id()
).dropDuplicates()


# write songplays table to parquet files partitioned by year and month
songplays_table.write.partitionBy(
    "year", "month").mode(
    "overwrite").parquet(output_data + "/songplays_table")